In [ ]:
import pandas as pd
# from time import process_time, sleep
import numpy as np
import torch
from torch_geometric.data import InMemoryDataset, download_url
import networkx as nx
from networkx.algorithms import community
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from pyvis.network import Network
import string
from colorama import Fore
import termtables as tt
string.punctuation
import seaborn as sns

In [ ]:
d_conclusion=pd.read_csv('Insight_dataset.csv',index_col=0)
d_conclusion

# Citation net

In [ ]:
# Edge dataframe
columns = ['sourceNodeId','targetNodeId','cite_info','relationshipType','flag']
d_edges=pd.DataFrame(columns=columns)

In [ ]:
# Insert data to edge dataframe
count=0
for i in range(len(d_conclusion)):
    try:
        List=eval(d_conclusion['ref_paper_id'][i])
        # cite_dic=eval(d_conclusion['cite_text'][i])
        count=count+1
    except:
        continue
    for j in List:
        if j in list(d_conclusion['corpusid']):
            d_edges_append=pd.DataFrame(data=[[d_conclusion['corpusid'][i],j,str(List),'cites',count]], columns=columns)
            d_edges=pd.concat([d_edges, d_edges_append], ignore_index=True, axis=0)

In [ ]:
d_edges

In [ ]:
d_edges.to_csv('d_edges.csv')

In [ ]:
# Corpus_id_list of papers 
total_id=list(set(d_edges['sourceNodeId']))
total_id.extend(list(set(d_edges['targetNodeId'])))

In [ ]:
print(len(set(total_id)))
total_id=list(set(total_id))

In [ ]:
#Corpus_id -> index -> add content 
re_num={}
for i in range (len(total_id)):
    content={}
    content['num']=i
    content['type']=1
    for j in range(len(d_conclusion)):
        if d_conclusion['corpusid'][j]==total_id[i]:
            # content['author-date']=total[total_id[i]]
            content['title']=d_conclusion['paper-title'][j]
            content['pdfurl']=d_conclusion['pdfurl'][j]
            content['conclusion']=d_conclusion['Content'][j]
            content['Solved']=d_conclusion['Solved'][j]
            content['Unsolved']=d_conclusion['Unsolved/FW'][j]

            if 'a survey' in content['title'] or 'A survey' in content['title'] or 'a Survey' in content['title'] or 'A Survey' in content['title']:
                content['type']=0
            else:
                content['type']=1
            
            # print('true')
            break
        
    re_num[total_id[i]]=content
re_num

In [ ]:
import json
with open("Net-content.json","w", encoding='utf-8') as f: 
    f.write(json.dumps(re_num,ensure_ascii=True))  

In [ ]:
# List of Source,Target
list_source=[]
list_target=[]
for i in range(len(d_edges)):
    if d_edges['sourceNodeId'][i]==d_edges['targetNodeId'][i]:
        continue
   
    list_source.append(re_num[d_edges['sourceNodeId'][i]]['num'])
    list_target.append(re_num[d_edges['targetNodeId'][i]]['num'])

In [ ]:
# Data Network 
class Demo_Data(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
    #file
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
   
    @property
    def processed_file_names(self):
        return ['data.pt']
    # download from website
    # def download(self):
    #     # Download to `self.raw_dir`.
    #     download_url(url, self.raw_dir)
        ...
    #data generate
    def process(self):
        # Read data into huge `Data` list.
        # Read data into huge `Data` list.
        # building
        
        start=0
        end=len(list_source)
        
        Edge_index = torch.tensor([list_source,
                                   list_target], dtype=torch.long)
        
        # s=list_source[start:end].copy()
        # s.extend(list_target[start:end])
        print(len(re_num))
        
        # node_num=len(set(s))

        # node feature
        sample_feature=[0]*len(re_num)
        X = torch.tensor(np.array(sample_feature).reshape(-1, 1).tolist(), dtype=torch.float)
        # node label
        # Y = torch.tensor([0,1,0],dtype=torch.float)
        
        #building
        # data = Data(x=X, edge_index=Edge_index, y=Y)
        data = Data(x=X,edge_index=Edge_index)
        
        # put in datalist
        data_list = [data]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        data.num_nodes = len(re_num)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
dataset = Demo_Data("Demo_Data_All")
data=dataset[0]
dataset.get_summary()

In [ ]:
# Display 
G = to_networkx(data, to_undirected=False)# Directed
degrees = [val for (node, val) in G.degree()]
display(pd.DataFrame(pd.Series(degrees).describe()).transpose().round(2))
print(len(degrees))
print(sum(degrees))
plt.figure(figsize=(10, 6))
# plt.hist(degrees, bins=100)
# plt.hist(degrees, range(0,10))
plt.hist(degrees, range(0,50),rwidth=0.5,align="left")

plt.xlabel("node degree")
plt.show()

In [ ]:
#Number of citations
ancestors={}
ancestors_num=[]
for i in G.nodes:
    # print(i)
    # print(nx.ancestors(G, i))
    ancestors[i]=len(nx.ancestors(G, i))
    ancestors_num.append(len(nx.ancestors(G, i)))

In [ ]:
#Papers of top 10 Citations
r_node_list=[]
while i<=10: 
    print('rank:',i)
    r_node=ancestors_num.index(sorted(ancestors_num,reverse=True)[0])
    print('high_contribution_node_num:',r_node)
    r_node_list.append(r_node)
    print('number of cited:',len(nx.ancestors(G, r_node)))
    print('cited_by:',nx.ancestors(G, r_node))
    G.nodes[r_node]['color']='deeppink'
    G.nodes[r_node]['shape']='triangle'
    G.nodes[r_node]['size']=35
    ancestors_num[r_node]=0 # clear
    i=i+1

In [ ]:
#Make graph - total 
word_node_num=len(total_id)
for i in re_num:
    #paper node shape,word,edge
    if re_num[i]['type']==0:
        G.nodes[re_num[i]['num']]['shape']='star'
        G.nodes[re_num[i]['num']]['color']='chocolate'
        # G.nodes[re_num[i]['num']]['label']='S'+str(re_num[i]['num'])
        G.nodes[re_num[i]['num']]['size']=50

    #paper label title
    try:
        # G.nodes[re_num[i]['num']]['label']=re_num[i]['author-date']
        G.nodes[re_num[i]['num']]['title']=re_num[i]['title']+'\n'+re_num[i]['pdfurl']+'\n'+str(re_num[i]['num'])
    except:
        continue

    word_node_num=word_node_num+4

net = Network(notebook=True, cdn_resources='in_line',directed=True,bgcolor='#fffefe')
net.from_nx(G)
net.toggle_physics(False)
net.show_buttons()
net.show('Cite_net_total.html')